In [32]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from dash import Dash, dcc, html, Input, Output
from plotly.subplots import make_subplots ### later delete

In [33]:
partners = pd.read_csv("rawdata/imf_import_export.csv")
partners = partners.iloc[: , 1:]
partners

,from,to,2019,2020
0,AF,W00,863.8,776.7
1,AF,XR43,841.1,758.0
2,AF,1C_092,772.8,699.0
3,AF,XS25,442.4,462.2
4,AF,IN,410.1,405.9
...,...,...,...,...
32250,W00,TV,90.7,79.4
32251,W00,NR,50.1,60.0
32252,W00,VA,31.7,71.5
32253,W00,MS,24.2,21.8


In [34]:
country_code = pd.read_csv('rawdata/imf_import_export_country_codes.csv', sep=';', names=['CountryCode', 'CountryName'])
country_code = country_code.iloc[:228, :]
partners = partners.merge(country_code, how ='right', left_on='from', right_on='CountryCode').drop(['CountryCode','from'], axis = 1)
partners.rename(columns={'CountryName':'from_name'}, inplace = True)
partners = partners.merge(country_code, how ='right', left_on='to', right_on='CountryCode').drop(['CountryCode','to'], axis = 1)
partners.rename(columns={'CountryName':'to_name'}, inplace = True)
partners

,2019,2020,from_name,to_name
0,0.7,23.2,Australia,Afghanistan
1,9.4,12.0,Austria,Afghanistan
2,82.7,37.7,Azerbaijan,Afghanistan
3,0.3,0.2,Bahrain,Afghanistan
4,4.3,6.4,Bangladesh,Afghanistan
...,...,...,...,...
24302,698.3,442.7,United Arab Emirates,Zambia
24303,84.4,71.3,United Kingdom,Zambia
24304,99.3,71.8,United States,Zambia
24305,0.4,0.7,Uruguay,Zambia


In [38]:
partners_jp_ex = partners[partners["from_name"]=="Japan"].sort_values("2019",ascending=False).head(10)

In [39]:
col1 = ['Label', 'Color']
col2 = ['Source','Target','Value','LinkColor']
node_df_2019 = pd.DataFrame(columns = col1)
link_df_2019 = pd.DataFrame(columns = col2)
node_df_2020 = pd.DataFrame(columns = col1)
link_df_2020 = pd.DataFrame(columns = col2)

for idx, (before, after, _, importer) in enumerate(partners_jp_ex.itertuples(index=False)):
    node_df_2019 = node_df_2019.append({'Label':importer, 'Color':'lightblue'},ignore_index=True)
    node_df_2020 = node_df_2020.append({'Label':importer, 'Color':'lightblue'},ignore_index=True)
    link_df_2019 = link_df_2019.append({'Source':idx, 'Target':10, 'Value': before, 'LinkColor':'lightblue'},ignore_index=True)
    link_df_2020 = link_df_2020.append({'Source':10, 'Target':idx, 'Value': after, 'LinkColor':'lightblue'},ignore_index=True)

node_df_2019 = node_df_2019.append({'Label':'Export', 'Color':'lightblue'},ignore_index=True)


In [40]:
partners_jp_im = partners[partners["to_name"]=="Japan"].sort_values("2019",ascending=False).head(10)

for idx, (before, after,exporter, _) in enumerate(partners_jp_im.itertuples(index=False)):
    if exporter not in node_df_2019["Label"].values:
        node_df_2019 = node_df_2019.append({'Label':exporter, 'Color':'pink'},ignore_index=True)
    if exporter not in node_df_2020["Label"].values:
        node_df_2020 = node_df_2020.append({'Label':exporter, 'Color':'pink'},ignore_index=True)
    
    source_2019 = node_df_2019[node_df_2019["Label"]==exporter].index.item()
    source_2020 = node_df_2020[node_df_2020["Label"]==exporter].index.item()
    link_df_2019 = link_df_2019.append({'Source':source_2019, 'Target':13, 'Value': before, 'LinkColor':'pink'},ignore_index=True)
    link_df_2020 = link_df_2020.append({'Source':13, 'Target':source_2020, 'Value': after, 'LinkColor':'pink'},ignore_index=True)

node_df_2019 = node_df_2019.append({'Label':'Import', 'Color':'pink'},ignore_index=True)


In [45]:
for i, _, target, _, _ in link_df_2020.itertuples():
    link_df_2020.iloc[i, 1] = target + 14

In [46]:
fig = go.Figure(data=[go.Sankey(
    node = dict(
      pad = 15,
      thickness = 5,
      line = dict(color = "black", width = 0.5),
      label = node_df_2019['Label'].to_list() + node_df_2020['Label'].to_list(),
      color = node_df_2019['Color'].to_list() + node_df_2020['Color'].to_list()
    ),
    link = dict(
      source = link_df_2019['Source'].to_list() + link_df_2020['Source'].to_list(), # indices correspond to labels, eg A1, A2, A1, B1, ...
      target = link_df_2019['Target'].to_list() + link_df_2020['Target'].to_list(),
      value =  link_df_2019['Value'].to_list() + link_df_2020['Value'].to_list(),
      color = link_df_2019['LinkColor'].to_list() + link_df_2020['LinkColor'].to_list()
  ))])

fig.update_layout(title_text="Japan Sankey Diagram", font_size=10)
fig.add_annotation(text="2019",
                  xref="paper", yref="paper",
                  x=0.3, y=1.12, showarrow=False)
fig.add_annotation(text="2020",
                  xref="paper", yref="paper",
                  x=0.75, y=1.12, showarrow=False)

fig.update_layout(
    autosize=False,
    width=700,
    height=700,
    margin=dict(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
    )
)
fig.show()

In [47]:
# Practice
fig = go.Figure(data=[go.Sankey(
    node = dict(
      pad = 15,
      thickness = 5,
      line = dict(color = "black", width = 0.5),
      label = ["US", "UK", "Export", "Import", "US", "UK"],
      color = ["black", "black", "lightblue", "pink", "black", "black"]
    ),
    link = dict(
      source = [0, 1, 0, 1, 2, 2, 3, 3], # indices correspond to labels, eg A1, A2, A1, B1, ...
      target = [2, 3, 3, 2, 4, 5, 4, 5],
      value =  [8, 4, 2, 2, 2, 4, 2, 8],
      color = [
                    "lightblue",
                    "pink",
                    "pink",
                    "lightblue",
                    "lightblue",
                    "lightblue",
                    "pink",
                    "pink"]
  ))])

fig.update_layout(title_text="Basic Sankey Diagram", font_size=10)
fig.add_annotation(text="2019",
                  xref="paper", yref="paper",
                  x=0.2, y=1, showarrow=False)
fig.add_annotation(text="2020",
                  xref="paper", yref="paper",
                  x=0.75, y=1, showarrow=False)
fig.show()